In [ ]:
# UNCOMMENT TO RUN THIS NOTEBOOK
# pip install nbformat pandas plotly requests

import plotly.graph_objs as go
import pprint as pp
import requests

from dotenv import load_dotenv
from physrisk.container import Container
from plotly.subplots import make_subplots
from typing import NamedTuple
import plotly.io

plotly.io.renderers.default = "notebook"
import sys
sys.path.append("../../..")
import physrisk
from physrisk.vulnerability_models.config_based_model import config_items_from_csv


# Vulnerability configuration

## Types of curve

Two types:
1) *Damage curves*  
For acute risks, this specifies the fractional damage or disruption (e.g. downtime or efficiency loss), given the occurrence of a hazard with a given indicator value.  
Most commonly, curve_type: 'indicator/piecewise_linear'.  
e.g. for riverine flood, the curve $x$ values are depth in metres and the curve $y$ values are damage as fraction of total insurable value.

2) *Threshold curves*  
For chronic risks, this specifies the fractional damage or disruption that occurs for hazard indicator threshold values. This is for hazard indicators that give the mean amount of time per year for which an indicator is greater than a threshold value.   
Most commonly, curve_type: 'threshold/piecewise_linear'.  
e.g. for chronic heat, the indicator might be the number of days per year for which the maximum near-surface air temperature is greater than $t$ degrees, $t$ being the threshold values. The curve $x$ values are the thresholds in degrees Celsius and the curve $y$ values are the losses in production that occur for each day for which the maximum temperature is at that value.

## Identifying assets
Assets are identified by:
- asset class (the broad class e.g. real estate asset or power generating asset within which the asset vulnerabilities are modelled in a similar way)
- a number of asset attributes

### In configuration 
The vulnerability function is identified by a set of attribute key/value pairs in the form '<attribute1_name>=<attribute1_value,<attribute2_name>=<attribute2_value>,...'. The attributes are matched to those of the asset. 

### Example 1: assets specified by type/location
We can define sets of functions by type and location, e.g. for real estate assets 'type=Buildings/Commercial,location=Europe'

### Example 2: identifier only
For families of vulnerability functions, we can define a set of curves by, e.g. 'hazus_id=43'.

In all cases, logic in code maps an asset's attributes onto the attributes that define the vulnerability function.


In [23]:
config_items = config_items_from_csv("/Users/joemoorhouse/Code/main/bnpp/physrisk/src/physrisk/data/vulnerability_config.csv")
c1 = next(c for c in config_items if c.asset_class == "RealEstateAsset" and c.indicator_id=="flood_depth")
c2 = next(c for c in config_items if c.asset_class == "ManufacturingAsset" and c.indicator_id=="days_wbgt_above")

In [27]:
fig1 = make_subplots(rows=1, cols=2)
fig1.add_trace(
    go.Scatter(x=c1.points_x, y=c1.points_y, name="Example damage curve"),
    row=1,
    col=1,
)
fig1.add_trace(
    go.Scatter(x=c2.points_x, y=c2.points_y, name="Example threshold curve"),
    row=1,
    col=2,
)
fig1.update_xaxes(
    title="Flood depth (m)", title_font={"size": 14}, row=1, col=1
)
fig1.update_xaxes(
    title="Threshold (°C)", title_font={"size": 14}, row=1, col=2
)
fig1.update_yaxes(title="Damage as fraction of insurable value", title_font={"size": 14}, row=1, col=1)
fig1.update_yaxes(title="Fractional loss of production", title_font={"size": 14}, row=1, col=2)

In [ ]:

from dataclasses import field
from typing import Optional, Sequence


class VulnerabilityConfigItem:
    hazard_class: str  # Name of the physrisk hazard class, e.g. 'RiverineInundation'. This can be a comma-separated list, e.g. 'RiverineInundation,CoastalInundation'.
    asset_class: str  # Name of physrisk asset class, e.g. 'RealEstateAsset'.
    asset_identifier: str # Identifier of the asset, within the class. This is a comma-separated list of key value pairs, e.g. 'type=Buildings/Commercial,location=Europe'. 
    indicator_id: str # The identifier of the hazard indicator, e.g. 'flood_depth'.
    indicator_units: Optional[str] # The units of the hazard indicator, e.g. 'metres'.
    impact_id: str # Whether the impact is a 'damage' or 'disruption'.
    impact_units: Optional[str] # The units of the impact, typically omitted.
    curve_type: str # Commonly 'indicator/piecewise_linear' or 'threshold/piecewise_linear'.
    points_x: Optional[Sequence[float]] # The vulnerability function 'x' values.
    points_y: Optional[Sequence[float]] # The vulnerability function 'y' values.
    points_z: Optional[Sequence[float]] # The vulnerability function 'z' values.
    cap_of_points_x: Optional[float] # Convenience parameter: if specified, caps the 'x' values.
    cap_of_points_y: Optional[float] # Convenience parameter: if specified, caps the 'y' values.
    activation_of_points_x: Optional[float] # Convenience parameter: if specified, the damage or disruption is zero below the activation threshold.

In csv, the lines look like (note the "" around comma-separated items):

Wind,Asset,"type=Generic,location=Asia",max_speed,m/s,damage,,indicator/piecewise_linear,"[20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,130]","[0,0,0,0.01,0.04,0.09,0.17,0.25,0.34,0.42,0.49,0.55,0.61,0.66,0.71,1.0]",,,



In terms of representing curves, I see (at least) 3 use-cases. Perhaps the most frequently-used case will be where we have the hazard intensity ($x$) values and the corresponding impact (damage/disruption) ($y$) values. This can be captured by $x$ and $y$ fields each containing an array. We can add a third $z$ field to capture information about the uncertainty in the vulnerability functions.

### Case 1: deterministic damage curve provided
The hazard intensity values, $x_i$, are given, $i \in [1 \dots n]$ and the corresponding impacts $y_i$.

$x = [x_1, x_2, \dots, x_n]$  
$y = [y_1, y_2, \dots, y_n]$  
$z$ is empty

### Case 2: mean and standard deviation provided
The hazard intensity values, $x_i$, are given, $i \in [1 \dots n]$.

$f_i(y) =  \mathbb{P}(Y=y|x_i)$      
$\mu_i = \int f_i(y) y  dy$   
$\sigma_i^2 = \int f_i(y) y^2  dy - \mu_i^2$

The means are given in $y$ and the standard deviations in $z$.

$x = [x_1, x_2, \dots, x_n ]$  
$y = [\mu_1, \mu_2, \dots, \mu_n ]$  
$z = [\sigma_1, \sigma_2, \dots, \sigma_n ]$

### Case 3: discrete piece-wise linear cumulative density function (CDF) provided
The hazard intensity values, $x_i$, are given, $i \in [1 \dots n]$.

$F_i(y) = \mathbb{P}(Y \leq y|x_i)$

The CDF, $F_i(y)$, is given for points $y_j$, $j \in [1 \dots m]$.
$F_{ij} = \mathbb{P}(Y \leq y_i|x_i)$

$x = [x_1, x_2, \dots, x_n ]$  
$y = [y_1, y_2, \dots, y_m ]$  
$z = [[F_{11}, F_{12}, \dots, F_{1m}], [F_{21}, F_{12}, \dots, F_{2m}], \dots, [F_{n1}, F_{n2}, \dots, F_{nm}]]$